<h2>Conditional Random Fields (CRF) model for Smartphones offer titles</h2>

In [1]:
import pandas as pd
import time

from sklearn import model_selection, metrics
from joblib import dump
from sklearn_crfsuite import metrics
import sklearn_crfsuite
import scipy
import sklearn

In [2]:
BIO_ENCODED_PRODUCT_TITLES_PATHFILE = "../_data/bio_encoded_product_titles.csv"

def get_bio_encoded_titles(file_path):
    '''
    Returns the features for the product titles (i.e. the words) as a list of 
    lists of strings and the labels for these features as a list of lists of strings.
    '''
    features = []
    labels = []
    classes = []

    bio_titles_df = pd.read_csv(BIO_ENCODED_PRODUCT_TITLES_PATHFILE, encoding='iso-8859-1')
    classes = bio_titles_df["BIOTag"].unique()

    for titleNum in bio_titles_df["TitleNumber"].unique():
        title_features = bio_titles_df.loc[bio_titles_df["TitleNumber"] == titleNum, "Word"].tolist()
        title_labels = bio_titles_df.loc[bio_titles_df["TitleNumber"] == titleNum, "BIOTag"].tolist()

        features.append(title_features)
        labels.append(title_labels)

    return features, labels, classes

In [3]:
start_time = time.time()
bio_encoded_titles = get_bio_encoded_titles(BIO_ENCODED_PRODUCT_TITLES_PATHFILE)
elapsed_time = round(time.time() - start_time, 3)

print("BIO-encoded titles collected. Elapsed time (s): {}".format(elapsed_time))
print("Number of BIO-encoded titles collected: {}\n".format(len(bio_encoded_titles[0])))

BIO-encoded titles collected. Elapsed time (s): 196.054
Number of BIO-encoded titles collected: 57535



In [4]:
# Show and example of a BIO-encoded product title by BIOTagger
i = 3100
print("BIO-encoded title i = {} ".format(i))
print("Features = {}".format(str(bio_encoded_titles[0][i])))
print("Labels = {}".format(str(bio_encoded_titles[1][i])))

BIO-encoded title i = 3100 
Features = ['samsung', 'galaxy', 's7', 'edge', 'smartphone', '55', 'zoll', '139', 'cm', '32gb', 'interner', 'speicher']
Labels = ['B-BRAND', 'B-MODEL', 'I-MODEL', 'I-MODEL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [17]:
def word2features(product_title, i):
    '''
    A product title is received as a list of words (i.e. strings).
    '''
    word = product_title[i]

    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isalpha()': word.isalpha(),
        'word.containsdigit()': any(char.isdigit() for char in word),
        'word.containsNonAlphanumericChars()': not word.isalnum(),
    }
    
    # The word is not the beggining of a product title
    if i > 0:
        preceding_word = product_title[i-1]
        features.update({
            '-1:word.lower()': preceding_word.lower(),
            '-1:word.istitle()': preceding_word.istitle(),
            '-1:word.isupper()': preceding_word.isupper(),
        })
        
    # The word is the beginning of a product title
    else:
        features['BOT'] = True

    # The word is not the end of a product title
    if i < len(product_title) - 1:
        subsequent_word = product_title[i+1]
        features.update({
            '+1:word.lower()': subsequent_word.lower(),
            '+1:word.istitle()': subsequent_word.istitle(),
            '+1:word.isupper()': subsequent_word.isupper(),
        })
        
    # The word is not the end of a product title
    else:
        features['EOT'] = True

    return features


def title2features(product_title):
    return [word2features(product_title, i) for i in range(len(product_title))]

In [18]:
features = [title2features(title) for title in bio_encoded_titles[0]]
labels = bio_encoded_titles[1]
classes = bio_encoded_titles[2]

X_train, X_test, y_train, y_test = model_selection.train_test_split(features, labels, test_size=0.30, random_state=0)

In [19]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.01,
    max_iterations=200,
    all_possible_transitions=True
)

start_time = time.time()
crf.fit(X_train, y_train)
elapsed_time = round(time.time() - start_time, 3)

print("CRF model training finished. Elapsed time (s): {}".format(elapsed_time))

y_pred = crf.predict(X_test)

CRF model training finished. Elapsed time (s): 58.06


<h2>Evaluation</h2>

In [20]:
# Remove 'O'-tagged words, as most of the words will be 'O'-tagged
# and the results will look much better than what they actually are
labels = list(crf.classes_)
labels.remove('O')

print(metrics.flat_classification_report(y_test, y_pred, labels = labels))

              precision    recall  f1-score   support

     B-BRAND       1.00      1.00      1.00     17605
     B-MODEL       1.00      1.00      1.00     17261
     B-COLOR       1.00      1.00      1.00     14514
     I-MODEL       1.00      1.00      1.00      9249
       B-RAM       0.98      0.98      0.98       843
       I-RAM       0.96      0.98      0.97       520

   micro avg       1.00      1.00      1.00     59992
   macro avg       0.99      0.99      0.99     59992
weighted avg       1.00      1.00      1.00     59992



In [38]:
# Predict some examples
sample_titles = [
    "Apple iPhone 4s 4G 64GB BLUE",
    "Huawei Mate 20 Lite",
    "Huawei P20",
    "Samsung Galaxy SII 64GB 4GB RAM",
    "ZTE Blade V9 Smartphone (14,5cm (5,7 Zoll) Display, 32 GB interner Speicher, Android) Schwarz",
    "Xiaomi Redmi Note 5 64Gb Negro",
    "Huawei P Smart 2019 Aurora blau 6,21\" 64GB 3GB RAM Dual-SIM",
    "iPhone 7 32GB - Gold",
    "Apple iPhone 7 Plus 128 GB Silber",
    "Xiaomi Redmi Go - Smartphone (1 GB de RAM, 8 GB de ROM), Color Negro",
    "Smmartphone Xiaomi Redmi Go 5.0\" 1GB 8GB Dual SIM Azul",
    "HUAWEI P30 Pro 6GB + 128GB, Aurora",
    "Gigaset GS100 Smartphone lemon green",
]

for title in sample_titles:
    splitted_title = title.split()
    title_featured = title2features(splitted_title)
    # TODO preprocess title
    labels = crf.predict_single(title_featured)
    prob = crf.predict_marginals_single(title_featured)
    print(splitted_title)
    print(labels)
    print(prob)

['Apple', 'iPhone', '4s', '4G', '64GB', 'BLUE']
['B-BRAND', 'B-MODEL', 'I-MODEL', 'O', 'O', 'O']
[{'B-BRAND': 0.999999999998774, 'B-MODEL': 1.3924631279347706e-14, 'O': 1.1750836847213896e-12, 'B-COLOR': 1.8609341050946005e-14, 'I-MODEL': 7.657634262835998e-16, 'B-RAM': 4.7567398896955994e-17, 'I-RAM': 1.746591490634558e-14}, {'B-BRAND': 3.480722583675516e-10, 'B-MODEL': 0.9999617581376175, 'O': 3.8207448177357146e-05, 'B-COLOR': 3.399585618195031e-08, 'I-MODEL': 1.3908576546697788e-11, 'B-RAM': 1.7159782120270712e-13, 'I-RAM': 5.619606125808191e-11}, {'B-BRAND': 1.415127348856085e-06, 'B-MODEL': 6.028941808879005e-09, 'O': 0.4490239341489174, 'B-COLOR': 6.519690072541341e-06, 'I-MODEL': 0.5509667953379508, 'B-RAM': 1.1753066639634614e-06, 'I-RAM': 1.543601040599406e-07}, {'B-BRAND': 6.086489649294963e-06, 'B-MODEL': 9.485068077533934e-06, 'O': 0.9999114989920175, 'B-COLOR': 4.153815539363056e-06, 'I-MODEL': 6.830066786673081e-05, 'B-RAM': 4.599766196541793e-07, 'I-RAM': 1.499022954235

[{'B-BRAND': 0.9999998791872028, 'B-MODEL': 1.5543699398370155e-11, 'O': 1.1635051390763912e-07, 'B-COLOR': 1.8474468305180428e-09, 'I-MODEL': 2.45849590363819e-12, 'B-RAM': 7.0997769095090265e-12, 'I-RAM': 2.5897346476403736e-09}, {'B-BRAND': 8.547768770872883e-14, 'B-MODEL': 0.9999999917033333, 'O': 8.28193415926304e-09, 'B-COLOR': 6.199040818150572e-12, 'I-MODEL': 7.638456737319691e-12, 'B-RAM': 4.301962912118944e-15, 'I-RAM': 8.051486202315103e-13}, {'B-BRAND': 7.910158724539482e-07, 'B-MODEL': 2.507996740646833e-12, 'O': 0.000628732575434716, 'B-COLOR': 7.01489600152264e-08, 'I-MODEL': 0.999370406131393, 'B-RAM': 1.5861474181146784e-12, 'I-RAM': 1.2424585613486533e-10}, {'B-BRAND': 4.254902965029924e-06, 'B-MODEL': 0.000328369210987695, 'O': 0.9994552576597477, 'B-COLOR': 7.677562631987458e-05, 'I-MODEL': 0.00013503845921524995, 'B-RAM': 3.0406358113771793e-07, 'I-RAM': 7.718340242023275e-11}, {'B-BRAND': 6.021801886261884e-10, 'B-MODEL': 1.6828140071662156e-08, 'O': 0.99999995428

[{'B-BRAND': 0.9999998579708544, 'B-MODEL': 7.084456679122689e-10, 'O': 1.3651070546486178e-07, 'B-COLOR': 2.1650696497088264e-09, 'I-MODEL': 4.0223822494833046e-11, 'B-RAM': 7.105372531778135e-12, 'I-RAM': 2.597595367824889e-09}, {'B-BRAND': 1.314180222773893e-10, 'B-MODEL': 0.9999870048315591, 'O': 1.2985588958192529e-05, 'B-COLOR': 9.412311305690179e-09, 'I-MODEL': 1.4362322196057432e-11, 'B-RAM': 6.048556686309743e-14, 'I-RAM': 2.1330286138973958e-11}, {'B-BRAND': 8.512965079718484e-06, 'B-MODEL': 2.5550446958542003e-09, 'O': 0.9994885350237063, 'B-COLOR': 5.7098797521930285e-05, 'I-MODEL': 0.0004458076186590761, 'B-RAM': 1.2073741371993792e-09, 'I-RAM': 4.183261396108462e-08}, {'B-BRAND': 0.0012472106416660746, 'B-MODEL': 4.574876422220326e-05, 'O': 0.990729417148408, 'B-COLOR': 0.0077540694103513065, 'I-MODEL': 0.00010605082615776396, 'B-RAM': 0.00011750036496611397, 'I-RAM': 2.8442283108012687e-09}, {'B-BRAND': 3.8125040400148496e-09, 'B-MODEL': 2.7182177467053013e-08, 'O': 0.99

[{'B-BRAND': 0.4195448175020373, 'B-MODEL': 0.018984861171664723, 'O': 0.5515325516613871, 'B-COLOR': 0.008689288620887348, 'I-MODEL': 0.001032562215716456, 'B-RAM': 1.521681441702826e-07, 'I-RAM': 0.00021576666016268577}, {'B-BRAND': 3.3167138155903377e-06, 'B-MODEL': 0.01020252533612644, 'O': 0.9895745039585232, 'B-COLOR': 2.0151548807416586e-05, 'I-MODEL': 0.0001939858698258743, 'B-RAM': 4.964112425791554e-06, 'I-RAM': 5.524604754609538e-07}, {'B-BRAND': 1.7152736210594433e-08, 'B-MODEL': 1.2540507381331237e-09, 'O': 0.9999999482198171, 'B-COLOR': 2.1620399549763805e-08, 'I-MODEL': 1.1705047097597823e-08, 'B-RAM': 3.7660895661906286e-13, 'I-RAM': 4.7572625217038577e-11}, {'B-BRAND': 0.001527146211498813, 'B-MODEL': 0.0004694622661055813, 'O': 0.9977115406289109, 'B-COLOR': 0.00029111436956860893, 'I-MODEL': 7.182292266427642e-07, 'B-RAM': 1.8283305478989416e-08, 'I-RAM': 1.1383889889430425e-11}, {'B-BRAND': 2.1721321317646726e-05, 'B-MODEL': 7.645253912361497e-05, 'O': 0.99708106667

In [22]:
CRF_MODEL_OUTPUT_FILE = "../_models/crf.joblib"

# Dump CRF model to file
dump(crf, CRF_MODEL_OUTPUT_FILE)

['../_models/crf.joblib']

<h2>Observations</h2>